In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [23]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
##sample run
text = "Quote: We are standing on "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: We are standing on <strong>the</strong> <strong>edge</strong> of a new world. <strong>The</strong> <strong>edge</strong>


In [8]:
import pandas as pd
from datasets import Dataset
df = pd.read_csv('/kaggle/input/ts-lyrics/lyrics.csv')
df.head()

,track_name,album_name,lyrics,tokens,album_year,lexical_diversity
0,Tim McGraw,Taylor Swift,he said the way my blue eyes shined put those...,366,2006,0.333333
1,Picture To Burn,Taylor Swift,state the obvious i didnt get my perfect fanta...,292,2006,0.376712
2,Teardrops On My Guitar,Taylor Swift,drew looks at me i fake a smile so he wont se...,288,2006,0.447917
3,A Place In This World,Taylor Swift,i dont know what i want so dont ask me cause ...,232,2006,0.353448
4,Cold As You,Taylor Swift,you have a way of coming easily to me and whe...,245,2006,0.416327


In [10]:
dataset = Dataset.from_pandas(df,split='train')

In [12]:
dataset = dataset.rename_column("lyrics", "text")
dataset[0]

{'track_name': 'Tim McGraw',
 'album_name': 'Taylor Swift',
 'text': 'he said the way my blue eyes shined  put those georgia stars to shame that night  i said thats a lie  just a boy in a chevy truck  that had a tendency of gettin stuck  on backroads at night  and i was right there beside him all summer long  and then the time we woke up to find that summer gone  but when you think tim mcgraw  i hope you think my favorite song  the one we danced to all night long  the moon like a spotlight on the lake  when you think happiness  i hope you think that little black dress  think of my head on your chest  and my old faded blue jeans  when you think tim mcgraw  i hope you think of me  september saw a month of tears  and thankin god that you werent here  to see me like that  but in a box beneath my bed  is a letter that you never read  from three summers back  its hard not to find it all a little bittersweet  and lookin back on all of that its nice to believe  when you think tim mcgraw  i hop

In [13]:
dataset = dataset.remove_columns(["track_name","album_name","tokens","album_year","lexical_diversity"])
dataset[0]

{'text': 'he said the way my blue eyes shined  put those georgia stars to shame that night  i said thats a lie  just a boy in a chevy truck  that had a tendency of gettin stuck  on backroads at night  and i was right there beside him all summer long  and then the time we woke up to find that summer gone  but when you think tim mcgraw  i hope you think my favorite song  the one we danced to all night long  the moon like a spotlight on the lake  when you think happiness  i hope you think that little black dress  think of my head on your chest  and my old faded blue jeans  when you think tim mcgraw  i hope you think of me  september saw a month of tears  and thankin god that you werent here  to see me like that  but in a box beneath my bed  is a letter that you never read  from three summers back  its hard not to find it all a little bittersweet  and lookin back on all of that its nice to believe  when you think tim mcgraw  i hope you think my favorite song  the one we danced to all night

In [15]:
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 124
    })
    test: Dataset({
        features: ['text'],
        num_rows: 32
    })
})

In [16]:
dataset = dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [17]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [20]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
    text = f"Lyrics: {example['text'][0]}"
    return [text]

In [25]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    args=transformers.TrainingArguments(
        overwrite_output_dir= True,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="/kaggle/working/",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)
trainer.train()

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,0.511400
2,0.511400
3,0.504200
4,0.487100
5,0.470000
6,0.453600
7,0.438700
8,0.425900
9,0.415600
10,0.408300


TrainOutput(global_step=10, training_loss=0.4626451224088669, metrics={'train_runtime': 68.0301, 'train_samples_per_second': 1.176, 'train_steps_per_second': 0.147, 'total_flos': 40869483724800.0, 'train_loss': 0.4626451224088669, 'epoch': 10.0})

In [26]:
trainer.save_model('/kaggle/working/')

In [32]:
###Trying a inference
text = 'Lyrics: We were so in love'
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Lyrics: We were so in love, but you were so cold. I tried to be strong, but I couldn't hold. I tried to be brave, but I couldn't hide. I tried to be strong, but I couldn't hide. I tried to be brave, but I couldn't hide. I tried to be strong, but I couldn't hide. I tried to be brave, but I couldn't hide. I tried to be strong, but I couldn't hide. I tried
